In [1]:
# import libraries
import os
import random
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns   
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import sklearn.model_selection as model_selection

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

from collections import Counter
# from imblearn.over_sampling import SMOTE
#from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import linear_model
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

%matplotlib inline

In [14]:
#import consolidated saved csvs of wav features

train_wav = pd.read_csv('../data/df_train_wav_consolidated.csv',index_col=0)
test_wav = pd.read_csv('../data/df_test_wav_finc.csv',index_col=0)

#Get metadata
metadata=pd.read_csv('../data/musicnet_metadata.csv')

In [15]:
#Make a copy of the metadata
meta_data_copy = metadata.copy(deep=True)
meta_data_copy.reset_index(inplace=True)
#Rename column name
meta_data_copy = meta_data_copy.rename(columns = {'id':'filename'})

merged_train_data = pd.merge(train_wav , meta_data_copy , on="filename")
merged_train_data = merged_train_data.drop(["composer", "composition", "movement","source","transcriber","catalog_name","index"], axis=1)

merged_test_data = pd.merge(test_wav , meta_data_copy , on="filename")
merged_test_data = merged_test_data.drop(["composer", "composition", "movement","source","transcriber","catalog_name","index"], axis=1)

In [16]:
# #import midi file tables
# train_labels = pd.read_csv('../data/df_train_labels_consol.csv')
# test_labels = pd.read_csv('../data/df_test_labels_consol.csv')

# #combine train and test data
# labels_frames = [train_labels,test_labels]
# labels = pd.concat(labels_frames , ignore_index=True)

# #create series with various midi features

# #number of unique instruments
# midi_nunique_inst = labels.groupby('file_name').agg({"instrument": "nunique"}).reset_index().rename(columns={"instrument": "midi_nunique_inst"})

# #number of unique notes
# midi_nunique_note = labels.groupby('file_name').agg({"note": "nunique"}).reset_index().rename(columns={"note": "midi_nunique_note"})

# #total number of notes
# midi_num_notes = labels.groupby('file_name').sum()[['note']].reset_index().rename(columns={'note':'midi_num_notes'})

# #quintiles of the distribution of note values (pitches)
# midi_min_note = labels.groupby('file_name').min()[['note']].reset_index().rename(columns={'note':'midi_min_note'})
# midi_second_quintile_note = labels[['file_name','note']].groupby('file_name').quantile(q=0.25,interpolation='nearest')[['note']].reset_index().rename(columns={'note':'midi_second_quintile_note'})
# midi_median_note = labels[['file_name','note']].groupby('file_name').quantile(interpolation='nearest')[['note']].reset_index().rename(columns={'note':'midi_median_note'})
# midi_fourth_quintile_note = labels[['file_name','note']].groupby('file_name').quantile(q=0.75,interpolation='nearest')[['note']].reset_index().rename(columns={'note':'midi_fourth_quintile_note'})
# midi_max_note = labels[['file_name','note']].groupby('file_name').max()[['note']].reset_index().rename(columns={'note':'midi_max_note'})

# #average number of notes per instrument
# midi_avg_notes_inst = labels.groupby(['file_name','instrument']).sum()[['note']].groupby('file_name').mean().reset_index().rename(columns={'note':'midi_avg_notes_inst'})

# #merge all columns together
# midi_features = midi_nunique_inst.merge(midi_nunique_note, on='file_name')
# midi_features = midi_features.merge(midi_num_notes,on='file_name')
# midi_features = midi_features.merge(midi_min_note,on='file_name')
# midi_features = midi_features.merge(midi_second_quintile_note,on='file_name')
# midi_features = midi_features.merge(midi_median_note,on='file_name')
# midi_features = midi_features.merge(midi_fourth_quintile_note,on='file_name')
# midi_features = midi_features.merge(midi_max_note,on='file_name')
# midi_features = midi_features.merge(midi_avg_notes_inst,on='file_name')

# midi_features.to_csv('../data/midi_features.csv')

In [17]:
#Make a copy of the metadata
meta_data_copy = metadata.copy(deep=True)
meta_data_copy.reset_index(inplace=True)
#Rename column name
meta_data_copy = meta_data_copy.rename(columns = {'id':'filename'})

merged_train_data = pd.merge(train_wav , meta_data_copy , on="filename")
merged_train_data = merged_train_data.drop(["composer", "composition", "movement","source","transcriber","catalog_name","index"], axis=1)

merged_test_data = pd.merge(test_wav , meta_data_copy , on="filename")
merged_test_data = merged_test_data.drop(["composer", "composition", "movement","source","transcriber","catalog_name","index"], axis=1)

In [18]:
#subset of dataset for only max 2 instruments

subset_list= ['Accompanied Cello','Accompanied Clarinet','Accompanied Violin','Solo Cello', 'Solo Flute', 'Solo Piano', 'Solo Violin', 'Violin and Harpsichord']
subset_merged_train_data = merged_train_data.loc[merged_train_data['ensemble'].isin(subset_list)]
subset_merged_test_data = merged_test_data.loc[merged_test_data['ensemble'].isin(subset_list)]

In [19]:
#Get list of unique ensembles
ens_list = merged_train_data['ensemble'].unique()
ens_list

array(['Piano Quintet', 'Solo Piano', 'Piano Trio', 'Viola Quintet',
       'String Quartet', 'Clarinet Quintet',
       'Pairs Clarinet-Horn-Bassoon', 'Wind Quintet', 'Accompanied Cello',
       'Accompanied Clarinet', 'Wind and Strings Octet', 'String Sextet',
       'Piano Quartet', 'Horn Piano Trio', 'Solo Violin', 'Solo Flute',
       'Solo Cello', 'Violin and Harpsichord',
       'Clarinet-Cello-Piano Trio', 'Accompanied Violin', 'Wind Octet'],
      dtype=object)

In [20]:
#Map list of unique ensemble names to integer
mapping = {item:i for i, item in enumerate(ens_list)}

merged_train_data["ensemble"] = merged_train_data["ensemble"].apply(lambda x: mapping[x])
subset_merged_train_data["ensemble"] = subset_merged_train_data["ensemble"].apply(lambda x: mapping[x])

merged_test_data["ensemble"] = merged_test_data["ensemble"].apply(lambda x: mapping[x])
subset_merged_test_data["ensemble"] = subset_merged_test_data["ensemble"].apply(lambda x: mapping[x])

/Volumes/Kieran_External_HD/Users/Kieran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Volumes/Kieran_External_HD/Users/Kieran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
#This is the original train test split given in kaggle
X_original_train = merged_train_data.iloc[:,np.r_[:167,168]]
X_original_test = merged_test_data.iloc[:,np.r_[:167,168]]

y_original_train = merged_train_data.iloc[:,167:168]
y_original_test = merged_test_data.iloc[:,167:168]

In [31]:
#Concatenate the training and test data
x_frames = [X_original_train,X_original_test]
X = pd.concat(x_frames , ignore_index=True)

y_frames = [y_original_train,y_original_test]
y = pd.concat(y_frames , ignore_index=True)

#Find the index position of viola quintet and drop it since there is only one ensemble of that type
index_violaquintet = y[ y['ensemble'] == 3 ].index

y.drop(index_violaquintet , inplace=True)
X.drop(index_violaquintet , inplace=True)

###########################################ADDING MIDI FEATURES################################
#import and add midi features to input data
midi_features = pd.read_csv('../data/midi_features.csv')
midi_features.drop('Unnamed: 0',axis=1,inplace=True)

X_with_midi = X.merge(midi_features,left_on = 'filename',right_on='file_name')
X_with_midi.drop('filename',axis=1,inplace=True)

###############################################################################################

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, stratify=y, random_state=101)
X_train_with_midi, X_test_with_midi, y_train, y_test = train_test_split(X_with_midi, y, train_size=0.80, test_size=0.20, stratify=y, random_state=101)

In [ ]:
# X_with_midi.to_csv('../data/df_wav_and_midi.csv')

In [ ]:
X_train.drop('filename',axis=1,inplace=True) 
X_test.drop('filename',axis=1,inplace=True) 

In [ ]:
X_train.columns

In [ ]:
y_test.shape

## PCA Data Reduction

In [ ]:
help(f1_score)

In [ ]:
#all possible values of principal components can go up to the number of features
all_possible_k = range(1,len(X_train.columns))

# Create empty list to store fraction of total variance
all_possible_k_variance = []

# Write for loop to try k values for PCA
for i in all_possible_k:
    pca=PCA(n_components = i)
    pca.fit(X_train)
    train_pca = pca.transform(X_train)
    test_pca = pca.transform(X_test)
    
    #Append fractions of explained total variance
    all_possible_k_variance.append(np.sum(pca.explained_variance_ratio_))
    
    #Logistic Regression Model with PCA
    reg = linear_model.LogisticRegression(max_iter = 100,solver='liblinear')
    reg.fit(train_pca,y_train)
    reg_predict = reg.predict(test_pca)
    print("F1 Score with ", i," principal components: {:.2f}".format(f1_score(y_test,reg_predict,average='weighted')))

    np.set_printoptions(precision=2)

    


plt.plot(all_possible_k[0:8], all_possible_k_variance[0:8])
print(all_possible_k_variance[0:8])

# show lineplot of fraction of total variance vs. number of principal components, for all possible numbers of principal components.
# Add title and axis names
plt.title('Number of PCA Components vs Explained Variance Fraction ')
plt.xlabel('Number of PCA Components')
plt.ylabel('Explained Variance')

plt.show()  

In [ ]:
pca=PCA(n_components = 5)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

## Logistic Regression

### Logistic Model

In [ ]:
# help(linear_model.LogisticRegression)

In [ ]:
X_test.shape

In [ ]:
#Logistic Regression Model with PCA
reg = linear_model.LogisticRegression(max_iter = 100,solver='liblinear')
reg.fit(X_train_pca,y_train)
reg_predict = reg.predict(X_test_pca)
print(classification_report(y_test,reg_predict))

np.set_printoptions(precision=2)

# # Plot non-normalized confusion matrix
# titles_options = [("Confusion matrix, without normalization", None),
#                   ("Normalized confusion matrix", 'true')]
# for title, normalize in titles_options:
#     disp = plot_confusion_matrix(reg, X_test_pca, y_test,
#                                 #  display_labels=ens_list,
#                                  cmap=plt.cm.Blues,
#                                  normalize=normalize)
#     disp.ax_.set_title(title)
# plt.show()

In [ ]:
#Logistic Regression Model without PCA
reg = linear_model.LogisticRegression(max_iter = 100,solver='liblinear')
reg.fit(X_train,y_train)
reg_predict = reg.predict(X_test)
print(classification_report(y_test,reg_predict))

np.set_printoptions(precision=2)